# Cramer Rao Lower Bound
This notebook showcases how to get the Cramer Rao Lower Bound (CRLB) from a simulated dataset. 

In [1]:
%matplotlib qt 
#important for the em.MultiSpectrumVisualizer since this is an interactive plotting tool

In [2]:
from pyEELSMODEL.components.linear_background import LinearBG
from pyEELSMODEL.components.CLedge.zezhong_coreloss_edgecombined import ZezhongCoreLossEdgeCombined
from pyEELSMODEL.core.multispectrum import MultiSpectrumshape, MultiSpectrum
import pyEELSMODEL.api as em

import numpy as np
import matplotlib.pyplot as plt

## Simulation
A multispectrum will be simulated. Compared to the examples, the analytical model for the simulation should be known accuratly. For the example in Coreloss Example, MethodComparison and CeriumOxidationState, the background is a convolution between the low-loss and a power-low which is not an analytical function. \
In this example, the background is the sum of four power-laws, a carbon and oxygen edge are used (C/O=2) and no low loss is used since it is just to illustrate the use of the CRLB.

In [3]:
elements = ['C', 'O']
edges = ['K', 'K']
As = [1,2] #the relative amplitudes of the signals

E0 = 300e3 #V
alpha=1e-9 #rad
beta=20e-3 #rad

xsize = 100
ysize = 200

In [4]:
msh = MultiSpectrumshape(0.5, 200, 2048, xsize, ysize)
sh = msh.getspectrumshape()

In [5]:
bg = LinearBG(sh, rlist=np.linspace(1,5,4)) 
comp_elements = []
cte = 0.01 #constant to have a proper signal-to-background value

for elem, edge, A in zip(elements, edges, As):
    comp = ZezhongCoreLossEdgeCombined(sh, A*cte,E0,alpha,beta, elem, edge)
    comp_elements.append(comp)

mod = em.Model(sh, components=[bg]+comp_elements)
mod.calculate()

In [6]:
#shows the noisless spectrum
mod.plot()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Next cell creates a multispectrum of size: (xsize, ysize) and each row is the same spectrum with different noise. The noise is poissonian and this is changed by modifiying the total signal per spectrum (ne). Having a multispectrum where the noise content changes with respect to the columns but being equal for each row, gives us the ability to compare the CRLB with the found standard deviation as function of number of electrons. Moreover, a gain is defined which defines the number of counts created per electron. This information is important to have a proper estimate on the CRLB and depends on the detector used. 

In [7]:
gain = 5 #the signal one electron creates
eels_data = np.zeros((xsize, ysize, msh.Esize))
#ne is a exponential array to cover low to high signal-to-noise. 
ne = np.exp(np.linspace(1, 10, xsize)) #scaling factor to increase number of electrons per spectrum, hence improve signal-to-noise

for ii in range(xsize):
    ndata = np.copy(mod.data) * ne[ii]
    for jj in range(ysize):
        eels_data[ii,jj] = gain*np.random.poisson(ndata)

s = MultiSpectrum(msh, data=eels_data)
s.pppc = 1/gain

In [ ]:
em.MultiSpectrumVisualizer([s])

## Fitting Spectra

## Defining model and fitter
Since we know the groundth truth of the simulated multispectrum, we can define a model which will be used to fit it with. This model has three components:
1. Background: Four terms are used
2. Carbon K edge
3. Oxygen K edge

The fitter can be a linear one since the model is linear. Note that we optimize via the least squares method which is not exactly the same as the maximum likelihood estimator (MLE) for poisson noise. However, the linear least squares  and MLE are the same for gaussian noise which is approximatly valid.

In [8]:
bg = LinearBG(sh, rlist=np.linspace(1,5,4))

comp_elements = []
for elem, edge in zip(elements, edges):
    comp = ZezhongCoreLossEdgeCombined(sh, 1,E0,alpha,beta, elem, edge)
    comp_elements.append(comp)
    

mod = em.Model(sh, components=[bg]+comp_elements)
fit = em.LinearFitter(s, mod)


In [9]:
fit.multi_fit()

20000it [00:03, 5373.96it/s]


In [10]:
fig, maps, names = fit.show_map_result(comp_elements)

In [11]:
crlb_C = fit.CRLB_map(comp_elements[0].parameters[0]) #comp_elements[0].parameters[0]: amplitude of carbon edge
crlb_O = fit.CRLB_map(comp_elements[1].parameters[0]) #comp_elements[1].parameters[0]: amplitude of oxygen edge

20000it [00:11, 1788.38it/s]
20000it [00:11, 1794.87it/s]


In [16]:
fig, ax = plt.subplots(2,2)
ax[0,0].plot(ne/s.pppc, s.pppc*maps[0].mean(1)/ne, color='red')
ax[0,0].axhline(As[0]*cte, linestyle='dotted', color='black', label='Ground truth')
ax[0,0].set_ylabel('Average value/Total counts')
ax[0,0].set_title(r'Carbon abundance')

ax[0,1].plot(ne/s.pppc, s.pppc*maps[1].mean(1)/ne, color='blue')
ax[0,1].axhline(As[1]*cte, linestyle='dotted', color='black', label='Ground truth')
ax[0,1].set_title(r'Oxygen abundance')

ax[1,0].plot(ne/s.pppc, crlb_C.mean(1)/ne, color='black', label='CRLB')
ax[1,0].plot(ne/s.pppc, maps[0].std(1)/ne, color='red', label='std C')
# ax[1,0].set_xscale('log')
ax[1,1].plot(gain*ne, crlb_O.mean(1)/ne, color='black', label='CRLB')
ax[1,1].plot(gain*ne, maps[1].std(1)/ne, color='blue', label='std O')
# ax[1,1].set_xscale('log')
ax[1,0].set_ylabel('Standard deviation/Total counts')
ax[1,0].set_xlabel(r'Total counts')
ax[1,1].set_xlabel(r'Total counts')

for axe in ax.flatten():
    axe.set_xscale('log')
    axe.legend()